In [4]:
import sys
import shutil
at_colab = "google.colab" in sys.modules
if at_colab:
  !sudo apt install libglpk-dev python3.8-dev libgmp3-dev
  !apt-get install -y -qq glpk-utils
  !apt-get install -y -qq coinor-cbc


  if not shutil.which("pyomo"):
    !pip install -q pyomo
    assert(shutil.which("pyomo"))

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  libamd2 libbtf1 libcamd2 libccolamd2 libcholmod3 libcolamd2 libcxsparse3
  libglpk40 libgmp-dev libgmpxx4ldbl libgraphblas1 libklu1 libldl2 libmetis5
  libpython3.8 libpython3.8-dev libpython3.8-minimal libpython3.8-stdlib
  librbio2 libspqr2 libsuitesparse-dev libsuitesparseconfig5 libumfpack5
  python3.8 python3.8-minimal
Suggested packages:
  libiodbc2-dev gmp-doc libgmp10-doc libmpfr-dev python3.8-venv binfmt-support
The following NEW packages will be installed:
  libamd2 libbtf1 libcamd2 libccolamd2 libcholmod3 libcolamd2 libcxsparse3
  libglpk-dev libglpk40 libgmp-dev libgmp3-dev libgmpxx4ldbl libgraphblas1
  libklu1 libldl2 libmetis5 libpython3.8 libpython3.8-dev libpython3.8-minimal
  libpyth

In [5]:
import sys
if 'google.colab' in sys.modules:
    import os
    from google.colab import files
    # just check if we already uploaded, may we restart the runtime and run all cells
    if not os.path.isfile('Data Set Feedcalculator.xlsx'):
        uploaded = files.upload()

Saving Data Set Feedcalculator.xlsx to Data Set Feedcalculator.xlsx


In [6]:
import pandas as pd
data = pd.read_excel('Data Set Feedcalculator.xlsx',sheet_name=None)

In [7]:
ingredients = data['Ingredient Database']
nutrient_rules = data['Nutrient Rules']
ingredient_rules = data['Ingredient Rules']

In [8]:
available_ingredients = ingredients[ingredients.Availability][['Name','Reference name','Price']+list(nutrient_rules.Nutrient)].set_index('Reference name')

In [9]:
nutrient_bounds = {nut :(lb if not pd.isna(lb) else 0,     
                           ub if not pd.isna(ub) else None) for nut,lb,ub in zip(nutrient_rules.Nutrient,nutrient_rules['Lower Bound'],nutrient_rules['Upper Bound'])}

In [10]:
ingredient_bounds = {ing : (lb if not pd.isna(lb) else 0,
                             ub if not pd.isna(ub) else None) for ing,lb,ub in zip(ingredient_rules.Ingredient,ingredient_rules['Lower Bound'],ingredient_rules['Upper Bound'])}

In [11]:
combined_ingredient_rules = [] 
set_of_availabe_ingredients = set(available_ingredients.index)
for c in ['Unnamed: '+str(i) for i in range(5,13)]:
    aux = ingredient_rules[[c]].dropna().values
    aux = [v[0] for v in aux]
    upperbound = aux[0]
    ingredients_in_rule = set(aux[1:]).intersection(set_of_availabe_ingredients)
    if ingredients_in_rule:
        combined_ingredient_rules.append((upperbound,ingredients_in_rule))
combined_ingredient_rules

[(0.17, {'cotton', 'sunflower'}),
 (0.35, {'soybeanexp', 'soybeanmeal'}),
 (0.3, {'barley'}),
 (0.25, {'maizebranhighq', 'maizebranlowq'}),
 (0.07, {'fish', 'fishlq'}),
 (0.08, {'mbmeal'}),
 (0.25, {'casfine', 'caswhole', 'tapbran'})]

In [12]:
import pyomo.environ as pyo

In [13]:
def minimize_cost(combined_ingredient_rules: list =combined_ingredient_rules, 
          ingredient_bounds: dict =ingredient_bounds,
          nutrient_bounds: dict =nutrient_bounds,
          available_ingredients: pd.DataFrame =available_ingredients) -> pyo.ConcreteModel:
  
  
  model = pyo.ConcreteModel()


  '''Initialize sets model.I with list of all available ingredients and model.N with all nutrients'''
  model.I = pyo.Set(initialize=available_ingredients.index)
  model.N = pyo.Set(initialize=list(nutrient_bounds.keys()))


  '''Set the lower and upper bounds for ingredients usage'''
  def ingredients_bounds(model: pyo.ConcreteModel, i: int) -> tuple:
    if i in ingredient_bounds:
      return (ingredient_bounds[i][0], ingredient_bounds[i][1])
    else:
      return (0, 1)


  '''Define variable indexed by ingredient with correspond bounds'''
  model.X = pyo.Var(model.I, within=pyo.NonNegativeReals, bounds=ingredients_bounds)


  '''ConstraintList with constraints on each nutrient(upper and lower)'''
  model.nutrient_bounds = pyo.ConstraintList()
  for n in model.N:
    model.nutrient_bounds.add(expr=pyo.quicksum(model.X[i] * available_ingredients.at[i, n] for i in model.I) <= nutrient_bounds[n][1])
    model.nutrient_bounds.add(expr=pyo.quicksum(model.X[i] * available_ingredients.at[i, n] for i in model.I) >= nutrient_bounds[n][0])


  '''Check whether we have found the 1kg of feed mix in total'''
  model.total_feed = pyo.Constraint(expr=pyo.quicksum(model.X[i] for i in model.I) == 1)


  '''ConstraintList for each ingredient combination mix which should be limited'''
  model.ingredient_combination_constraints = pyo.ConstraintList()
  for i in range(len(combined_ingredient_rules)):
    model.ingredient_combination_constraints.add(expr = pyo.quicksum(model.X[ingr] for ingr in list(combined_ingredient_rules[i][1])) <= combined_ingredient_rules[i][0])


  '''Define the objective to minimize the cost of feed mix'''
  model.cost = pyo.Objective(expr=pyo.quicksum(model.X[i]*available_ingredients["Price"][i] for i in model.I), sense=pyo.minimize)


  return model



In [14]:
solution1 = minimize_cost()
pyo.SolverFactory('cbc').solve(solution1)
solution1.pprint()
print(f'\nMinimum cost of feed mix: {solution1.cost()}')

4 Set Declarations
    I : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   26 : {'barley', 'blood', 'boneash', 'cotton', 'fish', 'fishlq', 'gnseeds', 'maize', 'maizebranhighq', 'maizebranlowq', 'mbmeal', 'sugars', 'soybeanexp', 'soybeanmeal', 'sunflower', 'sunflowerseeds', 'tapbran', 'caswhole', 'casfine', 'wheatbran', 'lysine', 'dl', 'ltryp', 'dicaph', 'shells', 'salt'}
    N : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   14 : {'oebr', 'cp', 'cfibre', 'staew', 'ca', 'na', 'opp', 'dlysp', 'dmetp', 'dmcp', 'dthrp', 'dtryp', 'dvalp', 'dargp'}
    ingredient_combination_constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    7 : {1, 2, 3, 4, 5, 6, 7}
    nutrient_bounds_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
  

In [15]:
def minimize_ingredients(combined_ingredient_rules: list =combined_ingredient_rules, 
          ingredient_bounds: dict =ingredient_bounds,
          nutrient_bounds: dict =nutrient_bounds,
          available_ingredients: pd.DataFrame =available_ingredients) -> pyo.ConcreteModel:
  
  
  model = pyo.ConcreteModel()


  '''Initialize sets model.I with list of all available ingredients and model.N with all nutrients'''
  model.I = pyo.Set(initialize=available_ingredients.index)
  model.N = pyo.Set(initialize=list(nutrient_bounds.keys()))


  '''Set the lower and upper bounds for ingredients usage'''
  def ingredients_bounds(model: pyo.ConcreteModel, i: int) -> tuple:
    if i in ingredient_bounds:
      return (ingredient_bounds[i][0], ingredient_bounds[i][1])
    else:
      return (0, 1)


  '''Define variables X indexed by ingredient with correspond bounds and binary variable Y'''
  model.X = pyo.Var(model.I, within=pyo.NonNegativeReals, bounds=ingredients_bounds)
  model.Y = pyo.Var(model.I, within=pyo.Binary)


  '''ConstraintList with constraints on each nutrient(upper and lower)'''
  model.nutrient_bounds = pyo.ConstraintList()
  for n in model.N:
    model.nutrient_bounds.add(expr=pyo.quicksum(model.X[i] * available_ingredients.at[i, n] for i in model.I) <= nutrient_bounds[n][1])
    model.nutrient_bounds.add(expr=pyo.quicksum(model.X[i] * available_ingredients.at[i, n] for i in model.I) >= nutrient_bounds[n][0])


  '''Check whether we have found the 1kg of feed mix in total'''
  model.total_feed = pyo.Constraint(expr=pyo.quicksum(model.X[i] for i in model.I) == 1)


  '''ConstraintList for each ingredient combination mix which should be limited'''
  model.ingredient_combination_constraints = pyo.ConstraintList()
  for i in range(len(combined_ingredient_rules)):
    model.ingredient_combination_constraints.add(expr = pyo.quicksum(model.X[ingr] for ingr in list(combined_ingredient_rules[i][1])) <= combined_ingredient_rules[i][0])


  '''ConstraintList to define model.Y variable depending on model.X'''
  model.y_definition = pyo.ConstraintList()
  for i in model.I:
    model.y_definition.add( expr= model.X[i] <= model.Y[i] )


  '''Define the objective to minimize the amount of ingredients'''
  model.min_ingredients = pyo.Objective(expr=pyo.quicksum(model.Y[i] for i in model.I), sense=pyo.minimize)
      

  return model


In [16]:
solution2 = minimize_ingredients()
pyo.SolverFactory('cbc').solve(solution2)
solution2.pprint()
print(f'\nMinimum number of ingredients: {solution2.min_ingredients()}')

5 Set Declarations
    I : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   26 : {'barley', 'blood', 'boneash', 'cotton', 'fish', 'fishlq', 'gnseeds', 'maize', 'maizebranhighq', 'maizebranlowq', 'mbmeal', 'sugars', 'soybeanexp', 'soybeanmeal', 'sunflower', 'sunflowerseeds', 'tapbran', 'caswhole', 'casfine', 'wheatbran', 'lysine', 'dl', 'ltryp', 'dicaph', 'shells', 'salt'}
    N : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   14 : {'oebr', 'cp', 'cfibre', 'staew', 'ca', 'na', 'opp', 'dlysp', 'dmetp', 'dmcp', 'dthrp', 'dtryp', 'dvalp', 'dargp'}
    ingredient_combination_constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    7 : {1, 2, 3, 4, 5, 6, 7}
    nutrient_bounds_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
  

In [18]:
def minimize_cost_and_ingredients(combined_ingredient_rules: list =combined_ingredient_rules, 
          ingredient_bounds: dict =ingredient_bounds,
          nutrient_bounds: dict =nutrient_bounds,
          available_ingredients: pd.DataFrame =available_ingredients) -> pyo.ConcreteModel:
  

  model = pyo.ConcreteModel()

  '''Initialize sets model.I with list of all available ingredients and model.N with all nutrients'''
  model.I = pyo.Set(initialize=available_ingredients.index)
  model.N = pyo.Set(initialize=list(nutrient_bounds.keys()))

  '''Define a mutable parameter to control the number of ingredients in the feed mix'''
  model.P = pyo.Param(mutable=True, initialize=5)


  '''Set the lower and upper bounds for ingredients usage'''
  def ingredients_bounds(model: pyo.ConcreteModel, i: int) -> tuple:
    if i in ingredient_bounds:
      return (ingredient_bounds[i][0], ingredient_bounds[i][1])
    else:
      return (0, 1)


  '''Define variables X indexed by ingredient with correspond bounds and binary variable Y'''
  model.X = pyo.Var(model.I, within=pyo.NonNegativeReals, bounds=ingredients_bounds)
  model.Y = pyo.Var(model.I, within=pyo.Binary)


  '''ConstraintList with constraints on each nutrient(upper and lower)'''
  model.nutrient_bounds = pyo.ConstraintList()
  for n in model.N:
    model.nutrient_bounds.add(expr=pyo.quicksum(model.X[i] * available_ingredients.at[i, n] for i in model.I) <= nutrient_bounds[n][1])
    model.nutrient_bounds.add(expr=pyo.quicksum(model.X[i] * available_ingredients.at[i, n] for i in model.I) >= nutrient_bounds[n][0])


  '''Check whether we have found the 1kg of feed mix in total'''
  model.total_feed = pyo.Constraint(expr=pyo.quicksum(model.X[i] for i in model.I) == 1)


  '''ConstraintList for each ingredient combination mix which should be limited'''
  model.ingredient_combination_constraints = pyo.ConstraintList()
  for i in range(len(combined_ingredient_rules)):
    model.ingredient_combination_constraints.add(expr = pyo.quicksum(model.X[ingr] for ingr in list(combined_ingredient_rules[i][1])) <= combined_ingredient_rules[i][0])


  '''ConstraintList to define model.Y variable depending on model.X'''
  model.y_definition = pyo.ConstraintList()
  for i in model.I:
    model.y_definition.add( expr= model.X[i] <= model.Y[i] )

  '''Constraint to ensure that the number of ingredients used is the same as the value provided in the mutable parameter'''
  model.number_of_ingredients = pyo.Constraint(expr=pyo.quicksum(model.Y[i] for i in model.I) == model.P)

  '''Define the objective to minimize the cost of the feed for a fixed number of ingredients'''
  model.cost_for_min_ingredients = pyo.Objective(expr=pyo.quicksum(model.X[i]*available_ingredients["Price"][i] for i in model.I), sense=pyo.minimize)   


  
  

  return model

In [19]:
solution3 = minimize_cost_and_ingredients()
pyo.SolverFactory('cbc').solve(solution3)
solution3.pprint()
print(f'\nMinimum cost for the minimum number of ingredients used: {solution3.cost_for_min_ingredients()}')

5 Set Declarations
    I : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   26 : {'barley', 'blood', 'boneash', 'cotton', 'fish', 'fishlq', 'gnseeds', 'maize', 'maizebranhighq', 'maizebranlowq', 'mbmeal', 'sugars', 'soybeanexp', 'soybeanmeal', 'sunflower', 'sunflowerseeds', 'tapbran', 'caswhole', 'casfine', 'wheatbran', 'lysine', 'dl', 'ltryp', 'dicaph', 'shells', 'salt'}
    N : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   14 : {'oebr', 'cp', 'cfibre', 'staew', 'ca', 'na', 'opp', 'dlysp', 'dmetp', 'dmcp', 'dthrp', 'dtryp', 'dvalp', 'dargp'}
    ingredient_combination_constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    7 : {1, 2, 3, 4, 5, 6, 7}
    nutrient_bounds_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
  

In [20]:
'''Define a function to evaluate the cost of the feed mix with fixed number of ingredients'''
def cost_on_ingr(ingr_num: int) -> int:
  solution3.P = ingr_num
  pyo.SolverFactory('cbc').solve(solution3)
  return solution3.cost_for_min_ingredients()

In [21]:
import plotly.express as px
import plotly.graph_objects as go

In [22]:
'''Create a data frame with number of ingredients as indexes and minimal cost for this number correspondingly'''
res=[]
for num_of_ingr in range(5, 14):
  res.append(cost_on_ingr(num_of_ingr))


lists = [res, list(range(5,14))]


df = pd.concat([pd.Series(x) for x in lists], axis=1)
df.columns = ["Price", "Number of Ingredients"]

'''Visualize data by plotting the minimum price as a function of the number of ingredients in the feed'''
fig = px.bar(df, x='Number of Ingredients', y='Price', title="Minimum cost of feed depending on the number of ingredients allowed to be used", text_auto=True )
fig.add_trace(go.Scatter( x=df['Number of Ingredients'], y=df['Price'], line_color='#000000', name='Min cost per ingredients used' ))
fig.show()
